In [80]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import unicodedata

In [81]:
cities = [['Istanbul','TR'],['Moscow','RU'],['London','GB'],['Saint Petersburg','RU'],['Berlin','DE'],
          ['Madrid','ES'],['Kyiv','UA'],['Rome','IT'],['Bucharest','RO'],['Paris','FR'],['Minsk','BY'],
          ['Vienna','AT']]

In [32]:
def get_city_population(city,country):
    url = ("https://en.wikipedia.org/wiki/"+city) 
    response = requests.get(url)
    time.sleep(2)                               #quick solution - I just added this in case of slow internet, it should be tested for 200 code but for this, it's okay
    soup = BeautifulSoup(response.content, "html.parser")
#     table_string = soup.find(text=re.compile('^Population\Z'))           #Looking for any text between any tags (I used \Z for end of string because $ includes \n (end of line) too)
    parent = soup.select_one('.mergedtoprow:-soup-contains("Population")')#Looking for element with class = 'mergedtoprow' and contains string 'Population'

#     parent = table_string.find_parent('tr')                              #data that we need is in several 'tr' elements that are siblings, and the string we found is child of the first one of those elements (that's why i am looking for it's parent)
    description = ['City']
    population = [f"{city},{country}"]
    
    population_string = parent.get_text("|",strip='True')                # First 'tr' always contains 'Population' string, I've added | as separator for easier splitting since some contain only 
                                                                         #'Population', some have '[2]' as reference, some have actual number for population and it always comes as last
    description.append(population_string.split('|')[0])                  #Taking first element after split will give me 'Population' string
    population_string2 = population_string.split('|')[-1]                #Taking last element after split will give me either 'Population','[5]'-referencies or '234,567'-actual population
    if(population_string2.startswith('[') | (population_string2 == 'Population')): #In case it gives me 'Population' or referencies
        population.append('')                                            #I will just append empty string where population number should be
    else:
        population.append(population_string2)                            # If it gives me actual number, I append number

    siblings = parent.next_siblings                                      # I am looking for all siblings of the first 'tr' element we found(the one with 'Population')
    for sib in siblings:
        if(sib['class'] != ['mergedrow']):                               #it is a long list but we only need the few ones after our first and they all have class='mergedrow'
            break                                                        #as soon as loop comes into class='mergedtoprow' it will break, as we don't need those elements
        for ch in sib.children:                                          #all 'tr' elements that we found have children that contain data we need 
            if(ch['class']==['infobox-label']):                          #'th' child with class= 'infobox-label' contains Description string (e.g. City, Estimate, Rural, density...)
                temp = ch.get_text(strip='True').replace('\xa0',' ').replace('•','')     #strip=True removes all whitespaces around text. Some unnecessary '\xa0' striings were found so I just replaced them with ''
                description.append(temp)                         
            elif(ch['class']==['infobox-data']):                         #'th' child with class= 'infobox-data' contains Population string (e.g. 12,506,238 or some explanations for Ranks or density)
                population.append(ch.get_text(strip='True').replace('\xa0',' '))
    population_table = pd.DataFrame({"Description":description,          #putting everything into table
                       "Population":population
                      })
    return population_table                                              #returning table

In [33]:
population_tables = []
for city in cities:
    population_tables.append(get_city_population(city[0].replace(' ','_'),city[1]))

In [69]:
final_tables =[]
for table in population_tables:
    table = table.transpose().reset_index().rename(columns={'index':'Variable'})
    table.columns = table.iloc[0]
    table = table.iloc[1: , :]
    table = table.drop('Description',axis=1)
    final_tables.append(table)

In [71]:
for table in final_tables:
    print(table)
    print('\n')

0         City Population MegacityMetropolitan municipality           Rank  \
1  Istanbul,TR                                   15,462,452  1st in Turkey   

0       Urban            Urban density           Metro density  
1  15,149,358  5,879/km2(15,230/sq mi)  2,894/km2(7,500/sq mi)  


0       City Population  Estimate(2018)[7]  Rank
1  Moscow,RU                    12,506,468   1st


0       City Population      Total[A]                  Density      Urban  \
1  London,GB             8,961,989[1]  5,666/km2(14,670/sq mi)  9,950,000   

0               Metro City of London Greater London  
1  14,257,962[4](1st)   8,706 (67th)      9,425,622  


0                 City Population  Estimate(2018)[6]
1  Saint_Petersburg,RU                     5,351,935


0       City Population  City/State   Urban[3]   Metro[4]
1  Berlin,DE              3,664,088  4,473,101  6,144,600


0       City Population Capital cityandMunicipality            Rank  \
1  Madrid,ES                              3,223,3